In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Conv2D,Activation,MaxPooling2D,Dense,Flatten,Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import os
import shutil
import tensorflow as tf
import json
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

In [2]:
train_path="/home/jupyter-iec_roadquality/jupyter-iec_roadquality/plants/DatasetPlants/train"
val_path="/home/jupyter-iec_roadquality/jupyter-iec_roadquality/plants/DatasetPlants/val"
batch_size=10
image_size=150
epochs=60

In [3]:
def load_data():
    data_train=ImageDataGenerator().flow_from_directory(train_path,
                                                     target_size=(image_size,image_size),
                                                     batch_size=batch_size,
                                                     shuffle=True,
                                                     class_mode='categorical')
    data_val=ImageDataGenerator().flow_from_directory(val_path,
                                                     target_size=(image_size,image_size),
                                                     batch_size=batch_size,
                                                     shuffle=True,
                                                     class_mode='categorical')
    return data_train,data_val

In [4]:
def create_model(input_shape):
    data_augmentation = keras.Sequential(
        [
            layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
            layers.RandomContrast(factor=0.1),
            layers.RandomZoom(height_factor=0.1, width_factor=0.1),  # Phóng to/thu nhỏ ngẫu nhiên ảnh tối đa 20%
        ])
    inputs = layers.Input(shape=input_shape)
    x = data_augmentation(inputs)
    scale_layer = keras.layers.Rescaling(scale=1 / 255.0, offset=-1)
    x=scale_layer(x)
    model =tf.keras.applications.EfficientNetB4(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    for layer in model.layers[-3:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    # Rebuild top
    
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.1
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(50, activation="softmax", name="pred")(x)
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    model.summary()
    return model

In [5]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [ ]:
def count_img():
    path_json_curent="./newPlants.json"
    with open(path_json_curent, 'r') as file:
            json_data= json.load(file)
    for folder in os.listdir(train_path):
        folder_path=os.path.join(train_path,folder)
        print(folder,len(os.listdir(folder_path)),json_data[folder])
count_img()

In [ ]:
data_train,data_val=load_data()

In [ ]:
model=create_model((image_size,image_size,3))

In [8]:
model.load_weights("./plants_EfficientNetB4_150.h5")
# model.compile(
#     optimizer=keras.optimizers.Adam(1e-4),  # Low learning rate
#         loss="categorical_crossentropy",
#         metrics=["accuracy"],
#     )
# model.save('plants_EfficientNetB4_model150.h5')

In [ ]:
hist = model.fit(data_train, epochs=epochs, validation_data=data_val)
model.save_weights("plants_EfficientNetB4_150.h5")
plot_hist(hist)